In [2]:
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
from datetime import datetime
from dateutil.parser import parse
from Module.Preprocessing import *
from Module.Model import *

## 데이터 불러오기
- ipo = IPO에 대한 간단한 데이터
- trading = 종가,시가 등등의 거래정보에 대한 데이터
- finance = 재무정보에 대한 데이터
- market = 시장지수에 대한 데이터
- stock = 상장주식수에 대한 데이터

In [4]:
ipo_kq = pd.read_csv("Data/KQ_IPO.csv")
trading_kq = pd.read_csv("d:/공모전/DATA/KQ_Trading.csv")
finance_kq= pd.read_csv("d:/공모전/DATA/finance_before.csv", encoding = "cp949")
market_kq = pd.read_csv("Data/KQ_Market.csv")
stock_kq = pd.read_csv("Data/StockNum.csv",thousands=',')

In [5]:
## 복사본 생성
ipo = ipo_kq.copy()
trading = trading_kq.copy()
finance = finance_kq.copy()
market = market_kq.copy()
stock = stock_kq.copy()

## 인덱스 전처리
> 종목명과 종목마다 가진 변수들을 멀티인덱스 설정

In [4]:
ipo = ipo_processing(ipo)
trading = FeatureDf_processing(trading)
stock = FeatureDf_processing(stock)
finance = FeatureDf_processing(finance)

## 파이낸스 정보를 독립변수로 투입
- MatchNew = finance에 있는 변수들을 독립변수로 투입
- MatchItem_per/pbr/ev = per,pbr,ev에 대한 정보를 계산식으로 변수 생성

In [5]:
Train_data = Matchitem_New(finance,ipo)

## IPO_Per 계산 공식
Train_data['IPO_PER'] = Train_data['공모가']/(Train_data['당기순이익']*1000/Train_data['상장주식수'])
## IPO_Pbr
Train_data['IPO_PBR'] = Train_data['공모가']/(Train_data['자본총계']*1000/Train_data['상장주식수'])
## IPO_EV/EBIDTA
Train_data['IPO_EV/EBITDA'] = ((Train_data['공모가']*Train_data['상장주식수'])+Train_data['순부채'])/(Train_data['EBITDA2']*1000)

Train_data['공모 시가총액'] = Train_data['공모가'] * Train_data['상장주식수']

In [6]:
Train_data = MatchItem_per(Train_data,'1개월 후 PER',1,trading,stock,finance)
Train_data = MatchItem_per(Train_data,'3개월 후 PER',3,trading,stock,finance)

Train_data = MatchItem_pbr(Train_data,'1개월 후 PBR',1,trading,stock,finance)
Train_data = MatchItem_pbr(Train_data,'3개월 후 PBR',3,trading,stock,finance)

Train_data = MatchItem_ev(Train_data,'1개월 후 EV/EBITDA',1,trading,stock,finance)
Train_data = MatchItem_ev(Train_data,'3개월 후 EV/EBITDA',3,trading,stock,finance)

In [7]:
## 사용하는 함수만 추출
use_col = ['상장유형','공모가','상장일','상장주식수','경쟁률','공모 시가총액','매출총이익률','영업이익률','ROE','유동비율','당좌비율','부채비율',\
           '이자보상배율','매출채권회전율','재고자산회전율','매출액증가율',\
           '영업이익증가율','IPO_PER','IPO_PBR','IPO_EV/EBITDA','1개월 후 PER','3개월 후 PER','1개월 후 PBR','3개월 후 PBR',\
           '1개월 후 EV/EBITDA','3개월 후 EV/EBITDA']

In [8]:
Train_data = Train_data[use_col]

# Train_data.to_csv("Train_data_2.csv",encoding = 'euc-kr')

# Train_data = pd.read_csv("d:/공모전/Train_data_2.csv")

# Train_data.set_index("종목명",inplace =True)

## 트레이딩 변수를 독립변수로 투입
> 비영업일에 해당하는 날짜 제외
- MatchItem_mean = 상장일부터 해당 월까지의 평균 값
- MatchItem_rotation = 해당 기간의 시가총액회전율

In [9]:
dating = pd.read_csv("d:/공모전/IPO/IPO/Data/날짜.csv")

date = dating.columns[68:]

trade = trading[date] ## 비영업일 제외한 것

In [10]:
test = Train_data.copy()

### 1개월 변수

In [11]:
test = MatchItem_mean(test,trade,'상장일','거래량회전율',\
                      '1개월 평균 거래량회전율',1)
test = MatchItem_mean(test,trade,'상장일','일중변동률',\
                      '1개월 평균 일중변동률',1)
test = MatchItem_mean(test,trade,'상장일','개인 매도수량 비중'\
                      ,'1개월 평균 개인 매도수량 비중',1)
test = MatchItem_mean(test,trade,'상장일','개인 매수수량 비중',\
                      '1개월 평균 개인 매수수량 비중',1)

test = MatchItem_rotation(test,trade,'상장일','거래대금',\
                           '시가총액','1개월 시가총액회전율',1)


### 3개월 변수

In [12]:
test = MatchItem_mean(test,trade,'상장일','거래량회전율',\
                      '3개월 평균 거래량회전율',3)
test = MatchItem_mean(test,trade,'상장일','일중변동률',\
                      '3개월 평균 일중변동률',3)
test = MatchItem_mean(test,trade,'상장일','개인 매도수량 비중'\
                      ,'3개월 평균 개인 매도수량 비중',3)
test = MatchItem_mean(test,trade,'상장일','개인 매수수량 비중',\
                      '3개월 평균 개인 매수수량 비중',3)

test = MatchItem_rotation(test,trade,'상장일','거래대금',\
                           '시가총액','3개월 시가총액회전율',3)



## 시장변수를 독립변수로 투입
- MatchItem_marketmoney = 특정 두 기간 사이의 시장회전율
- MatchItem_Market_1_3 = 1년전 대비 1달전의 시장지표
- MatchItem_Market_ipo = 상장일 대비 몇개월 후의 시장지표 
- MatchItem_interest = 상장일 기준 국고금리

In [13]:
market.set_index('Item Name ', inplace = True)

### 1년 전 대비 1달 전 

In [14]:
test = MatchItem_marketmoney(test,market,'1년전 대비 시장 회전율')
test = MatchItem_Market_1_3(test, market,'1년 전 대비 시장수익률' \
                            , '종가지수')
test = MatchItem_Market_1_3(test , market, \
                            '1년 전 대비 광의통화 증감률' , '광의통화')
test = MatchItem_Market_1_3(test , market,\
                            '1년 전 대비 고객예탁금 증감률' , '고객예탁금')


### 상장일 대비 1개월 후

In [15]:
test = MatchItem_marketmoney_ipo(test,market,'1달 후 시장 회전율',1)
test = MatchItem_Market_ipo(test, market,'1달 후  시장수익률' \
                            , '종가지수',1)
test = MatchItem_Market_ipo(test , market, \
                            '1달 후 광의통화 증감률' , '광의통화',1)
test = MatchItem_Market_ipo(test , market,\
                            '1달 후 고객예탁금 증감률' , '고객예탁금',1)


### 상장일 대비 3개월 후

In [16]:
test = MatchItem_marketmoney_ipo(test,market,'3달 후 시장 회전율',3)
test = MatchItem_Market_ipo(test, market,'3달 후  시장수익률' \
                            , '종가지수',3)
test = MatchItem_Market_ipo(test , market, \
                            '3달 후 광의통화 증감률' , '광의통화',3)
test = MatchItem_Market_ipo(test , market,\
                            '3달 후 고객예탁금 증감률' , '고객예탁금',3)
test = MatchItem_interest(test, market,'상장일 기준 국고3년시장금리' ,\
                          '국고3년시장금리')

## 종속변수 계산
- return_rate = 공모가 대비 1,3,6개월 수익률 계산
- rate_month_to_month = 1_3 수익률 , 3_6 수익률
- return_rate_new = 종가 대비 1,3,6개월 수익률 계산

In [17]:
test = return_rate(test,trading,'공모가 대비 1개월 수익률',1)
test = return_rate(test,trading,'공모가 대비 3개월 수익률',3)
test = return_rate(test,trading,'공모가 대비 6개월 수익률',6)
test = rate_month_to_month(test, trading, '1_3_수익률', 3, 1)
test = rate_month_to_month(test, trading, '3_6_수익률', 6, 3)
test = test.rename(columns = {'1달 후  시장수익률': '1달 후 시장수익률','3달 후  시장수익률':'3달 후 시장수익률'})

In [18]:
# test.to_csv("KQ_ALL.csv" , encoding = 'euc-kr')

## 종가 관련 변수 추가

In [22]:
# test = pd.read_csv("d:/공모전/KQ_ALL.csv")

# # test.set_index("종목명",inplace =True)

test = MatchItem_endper(test,'종가_PER',trading,stock,finance)
test = MatchItem_endpbr(test,'종가_PBR',trading,stock,finance)
test = MatchItem_endev(test,'종가_EV/EBITDA',trading,stock,finance)

test = return_rate_new(test,trading,'종가 대비 1개월 수익률',1)

test = return_rate_new(test,trading,'종가 대비 3개월 수익률',3)
test = return_rate_new(test,trading,'종가 대비 6개월 수익률',6)


In [24]:
# test = pd.read_csv("D:\공모전\IPO\IPO\CleanData/KQ_all.csv")

# test.set_index("종목명",inplace =True)

test['종가_PER'] = test['종가_PER']
test['종가_PBR'] = test['종가_PBR']
test['종가_EV/EBITDA'] = test['종가_EV/EBITDA']
test['종가 대비 1개월 수익률'] = test['종가 대비 1개월 수익률']
test['종가 대비 3개월 수익률'] = test['종가 대비 3개월 수익률']
test['종가 대비 6개월 수익률'] = test['종가 대비 6개월 수익률']

# test.to_csv("KQ_ALL_2.csv", encoding = 'euc-kr')